# Indoor localization

An indoor positioning system (IPS) is a system to locate objects or people inside a building using radio waves, magnetic fields, acoustic signals, or other sensory information collected by mobile devices. There are several commercial systems on the market, but there is no standard for an IPS system.

IPSes use different technologies, including distance measurement to nearby anchor nodes (nodes with known positions, e.g., WiFi access points), magnetic positioning, dead reckoning. They either actively locate mobile devices and tags or provide ambient location or environmental context for devices to get sensed.

According to the [report](https://www.marketsandmarkets.com/Market-Reports/indoor-positioning-navigation-ipin-market-989.html), the global indoor location market size is expected to grow from USD 7.11 Billion in 2017 to USD 40.99 Billion by 2022, at a Compound Annual Growth Rate (CAGR) of 42.0% during the forecast period. Hassle-free navigation, improved decision-making, and increased adoption of connected devices are boosting the growth of the indoor location market across the globe.

In this problem, you are going to use signals from seven different wi-fi access points to define in which room the user is located.

In [9]:
import pandas
import numpy as np
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

Loading the data and breaking it into training and cross-validation sets.

In [2]:
train_set = pandas.read_csv('train_set.csv')
cv_set = pandas.read_csv('cv_set.csv')

train_data = train_set[['wifi'+str(i) for i in range(1, len(train_set.columns) - 1)]]
train_labels = train_set['room']
cv_data = cv_set[['wifi'+str(i) for i in range(1, len(cv_set.columns) - 1)]]
cv_labels = cv_set['room']

In [3]:
print(train_data[:10])
print(train_labels[:10])


   wifi1  wifi2  wifi3  wifi4  wifi5  wifi6  wifi7
0    -68    -57    -61    -65    -71    -85    -85
1    -63    -60    -60    -67    -76    -85    -84
2    -61    -60    -68    -62    -77    -90    -80
3    -65    -61    -65    -67    -69    -87    -84
4    -61    -63    -58    -66    -74    -87    -82
5    -62    -60    -66    -68    -80    -86    -91
6    -65    -59    -61    -67    -72    -86    -81
7    -63    -57    -61    -65    -73    -84    -84
8    -66    -60    -65    -62    -70    -85    -83
9    -67    -60    -59    -61    -71    -86    -91
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: room, dtype: int64


In [4]:
print(cv_data[:10])
print(cv_labels[:10])

   wifi1  wifi2  wifi3  wifi4  wifi5  wifi6  wifi7
0    -64    -56    -61    -66    -71    -82    -81
1    -63    -65    -60    -63    -77    -81    -87
2    -64    -55    -63    -66    -76    -88    -83
3    -65    -60    -59    -63    -76    -86    -82
4    -67    -61    -62    -67    -77    -83    -91
5    -61    -59    -65    -63    -74    -89    -87
6    -63    -56    -63    -65    -72    -82    -89
7    -66    -59    -64    -68    -68    -97    -83
8    -67    -57    -64    -71    -75    -89    -87
9    -63    -57    -59    -67    -71    -82    -93
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: room, dtype: int64


### Training XGBoost regressor

In [10]:
param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'multi:softmax', 'n_estimators': 150, 'num_class': 4}
num_round = 50

In [11]:
data = xgb.DMatrix(train_data, label=train_labels - 1)

In [12]:
bst = xgb.train(param, data)

In [13]:
def eval_cv(model):
    data = xgb.DMatrix(cv_data, label=cv_labels-1)
    preds = model.predict(data)
    score = metrics.accuracy_score(cv_labels-1, preds)
#     report = metrics.classification_report(cv_labels-1, preds)
#     print("Accuracy: {}".format(score))
    return score

In [14]:
score = eval_cv(bst)

In [15]:
print("Accuracy: {}".format(score))

Accuracy: 0.9899244332493703


### Tuning hyperparameters

In [16]:
best_score = 0
best_param = {}
for d in range(3, 9):
    for est in [50, 100, 150, 200, 250]:
        for num in [50, 100, 150, 200]:
            for eta in [0.5, 0.75, 1]:
                param = {'max_depth':d, 'eta':eta, 'silent':1,
                         'objective':'multi:softmax', 'n_estimators': est, 
                         'num_class': 4}

                bst = xgb.train(param, data, num)
                score = eval_cv(bst)
                if score > best_score:
                    best_score = score
                    best_param = param
                    print("Best: {}".format(score))

Best: 0.9874055415617129
Best: 0.9899244332493703
Best: 0.9924433249370277
